In [ ]:
import numpy as np
import kalshi_python
from kalshi_python.models import *
import uuid

In [ ]:
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels
import pymc3 as pm

In [ ]:
config = kalshi_python.Configuration()
config.host = 'https://demo-api.kalshi.co/trade-api/v2'

In [ ]:
# this is a demo only account and has no cashable money on it, but you're welcome to fund it :)
kalshi_api = kalshi_python.ApiInstance(
    email='ianjoffe@uchicago.edu',
    password='Linear-Sketch-2024!',
    configuration=config,
)

In [ ]:
kalshi_api.get_markets(limit=1000).markets

In [ ]:
min_volume = 10
max_volume = 50

markets = [m for m in kalshi_api.get_markets(limit=1000).markets 
    if (m.volume > min_volume and m.volume < max_volume)]

In [ ]:
markets[1].ticker

In [ ]:
market_hisotry = kalshi_api.get_market_history(markets[1].ticker)
order_times = np.array([order.ts for order in market_hisotry.history])
order_times = order_times - min(order_times)
waiting_times = np.diff(order_times)
waiting_times

In [ ]:
# MLE fit for exponential distribution
expon_lambda_MLE = 1/np.mean(waiting_times)
expon_lambda_MLE

In [ ]:
# likelihood of data under exponential fit
expon_loglik = np.sum(np.log(stats.expon.pdf(waiting_times, expon_lambda_MLE)))
expon_loglik

In [ ]:
# MLE for double-gamma does not exist, so we estimate paramaters with Bayesian methods
with pm.Model() as model:
    # priors via method of moments
    lambda_rate = pm.Gamma('lambda', alpha=np.mean(waiting_times)**2 / np.var(waiting_times)
                           , beta=np.mean(waiting_times) / np.var(waiting_times))
    observation = pm.Exponential('obs', lam=lambda_rate, observed=waiting_times)
    trace = pm.sample(1000)

In [ ]:
# likelihood ratio test compared to null dist., double-gamma (avoids overdispersion)

In [ ]:
# are agents lambdas close together, or far apart?
# difficult question since orderer IDs are not accessible